<a href="https://colab.research.google.com/github/allesid/NN_learning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 26.8 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import lightning as L
import torch
import torch.nn as nn
import torch.nn.functional as F
import h5py
import numpy as np
import torch.utils.data as data
import os
# import argparse
import tqdm

print(os.getcwd())
os.chdir('/content/drive/MyDrive/ColabNotebooks/MLCup2023Weather/sources')
print(os.getcwd())


class PersistantModel(nn.Module):

    def __init__(self, out_seq_len=12):
        super().__init__()
        self.out_seq_len = out_seq_len

    def forward(self, X):
        output = torch.stack([X[:, -1] for _ in range(self.out_seq_len)], dim=1)
        return output


class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding, activation):
        super().__init__()

        if activation == 'tanh':
            self.activation = torch.tanh
        elif activation == 'relu':
            self.activation = torch.relu

        self.conv = nn.Conv2d(
            in_channels=in_channels + out_channels,
            out_channels=4 * out_channels,
            kernel_size=kernel_size,
            padding=padding
        )

    def forward(self, X, H_prev, C_prev):
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)
        input_gate = torch.sigmoid(i_conv)
        forget_gate = torch.sigmoid(f_conv)
        output_gate = torch.sigmoid(o_conv)
        C = forget_gate * C_prev + input_gate * self.activation(C_conv)
        H = output_gate * self.activation(C)
        return H, C


class ConvLSTM(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding, activation):
        super().__init__()
        self.out_channels = out_channels
        self.convLSTMCell = ConvLSTMCell(in_channels, out_channels, kernel_size, padding, activation)

    def forward(self, X):
        batch_size, seq_len, _, height, width = X.size()
        output = torch.zeros(batch_size, seq_len, self.out_channels, height, width, device=self.convLSTMCell.conv.weight.device)
        H = torch.zeros(batch_size, self.out_channels, height, width, device=self.convLSTMCell.conv.weight.device)
        C = torch.zeros(batch_size, self.out_channels, height, width, device=self.convLSTMCell.conv.weight.device)
        for time_step in range(seq_len):
            H, C = self.convLSTMCell(X[:, time_step], H, C)
            output[:, time_step] = H
        return output


class Seq2Seq(nn.Module):

    def __init__(
        self, num_channels, num_kernels, kernel_size, padding, activation, num_layers, out_seq_len
    ):
        super().__init__()
        self.out_seq_len = out_seq_len

        self.sequential = nn.Sequential()
        self.sequential.add_module(
            'convlstm1',
            ConvLSTM(
                in_channels=num_channels,
                out_channels=num_kernels,
                kernel_size=kernel_size,
                padding=padding,
                activation=activation
            )
        )
        for layer_index in range(2, num_layers + 1):
            self.sequential.add_module(
                f'convlstm{layer_index}',
                ConvLSTM(
                    in_channels=num_kernels,
                    out_channels=num_kernels,
                    kernel_size=kernel_size,
                    padding=padding,
                    activation=activation
                )
            )
        self.conv = nn.Conv2d(
            in_channels=num_kernels,
            out_channels=num_channels,
            kernel_size=kernel_size,
            padding=padding
        )

    def forward(self, X):
        batch_size, seq_len, num_channels, height, width = X.size()
        inputs = torch.zeros(
            batch_size, seq_len + self.out_seq_len - 1, num_channels, height, width,
            device=self.conv.weight.device
        )
        inputs[:, :seq_len] = X
        output = self.sequential(inputs)
        output = torch.stack([
            self.conv(output[:, index + seq_len - 1])
            for index in range(self.out_seq_len)
        ], dim=1)
        return output


class ConvLSTMModel(L.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = Seq2Seq(
            num_channels=1,
            num_kernels=32,
            kernel_size=(3, 3),
            padding=(1, 1),
            activation='relu',
            num_layers=1,
            out_seq_len=12
        )

    def forward(self, x):
        x = x.to(device=self.model.conv.weight.device)
        output = self.model(x)
        return output

    def training_step(self, batch):
        x, y = batch
        out = self.forward(x)
        out[y == -1] = -1
        loss = F.mse_loss(out, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4)
        return optimizer


class RadarDataset(data.Dataset):

    def __init__(self, list_of_files, in_seq_len=4, out_seq_len=12, mode='overlap', with_time=False):
        self.in_seq_len = in_seq_len
        self.out_seq_len = out_seq_len
        self.seq_len = in_seq_len + out_seq_len
        self.with_time = with_time
        self.__prepare_timestamps_mapping(list_of_files)
        self.__prepare_sequences(mode)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        data = []
        for timestamp in self.sequences[index]:
            with h5py.File(self.timestamp_to_file[timestamp]) as d:
                data.append(np.array(d[timestamp]['intensity']))
        data = np.expand_dims(data, axis=1)
        data[data == -1e6] = 0
        data[data == -2e6] = -1
        inputs = data[:self.in_seq_len]
        targets = data[self.in_seq_len:]
        if self.with_time:
            return (inputs, self.sequences[index][-1]), targets
        else:
            return inputs, targets

    def __prepare_timestamps_mapping(self, list_of_files):
        self.timestamp_to_file = {}
        for filename in list_of_files:
            with h5py.File(filename) as d:
                self.timestamp_to_file = {
                    **self.timestamp_to_file,
                    **dict(map(lambda x: (x, filename), d.keys()))
                }

    def __prepare_sequences(self, mode):
        timestamps = np.unique(sorted(self.timestamp_to_file.keys()))
        if mode == 'sequentially':
            self.sequences = [
                timestamps[index * self.seq_len: (index + 1) * self.seq_len]
                for index in range(len(timestamps) // self.seq_len)
            ]
        elif mode == 'overlap':
            self.sequences = [
                timestamps[index: index + self.seq_len]
                for index in range(len(timestamps) - self.seq_len + 1)
            ]
        else:
            raise Exception(f'Unknown mode {mode}')
        self.sequences = list(filter(
            lambda x: int(x[-1]) - int(x[0]) == (self.seq_len - 1) * 600,
            self.sequences
        ))


def prepare_data_loaders(train_batch_size=9, valid_batch_size=1, test_batch_size=1):
    train_dataset = RadarDataset([
        '../train/2021-01-train.hdf5', '../train/2021-03-train.hdf5', '../train/2021-04-train.hdf5',
        '../train/2021-06-train.hdf5', '../train/2021-07-train.hdf5', '../train/2021-09-train.hdf5',
        '../train/2021-10-train.hdf5', '../train/2021-12-train.hdf5'])
    valid_dataset = RadarDataset([
        '../train/2021-02-train.hdf5', '../train/2021-05-train.hdf5', '../train/2021-08-train.hdf5',
        '../train/2021-11-train.hdf5'
    ])
    test_dataset = RadarDataset(['../2022-test-public.hdf5'], out_seq_len=0, with_time=True)
    train_loader = data.DataLoader(train_dataset, batch_size=train_batch_size, num_workers=7, shuffle=True)
    valid_loader = data.DataLoader(valid_dataset, batch_size=valid_batch_size, num_workers=7, shuffle=False)
    test_loader = data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)
    return train_loader, valid_loader, test_loader


def evaluate_on_val(model, valid_loader):
    rmses = np.zeros((12,), dtype=float)
    for item in tqdm.tqdm(valid_loader):
        inputs, target = item
        output = model(inputs)
        rmses += np.sum((
            np.square(target.detach().numpy() - output.detach().numpy())
        ) * (target.detach().numpy() != -1), axis=(0, 2, 3, 4))
    rmses /= len(valid_loader)
    return np.mean(np.sqrt(rmses))


def process_test(model, test_loader, output_file='../output.hdf5'):
    model.eval()
    i = 0
    while True:
        if os.path.exists(output_file):
            i += 1
            ofc = output_file.removesuffix('.hdf5')
            output_file = f"{ofc}{str(i)}.hdf5"
        else:
            break
    for index, item in tqdm.tqdm(enumerate(test_loader)):
        (inputs, last_input_timestamp), _ = item
        output = model(inputs)
        with h5py.File(output_file, mode='a') as f_out:
            for index in range(output.shape[1]):
                timestamp_out = str(int(last_input_timestamp[-1]) + 600 * (index + 1))
                f_out.create_group(timestamp_out)
                f_out[timestamp_out].create_dataset(
                    'intensity',
                    data=output[0, index, 0].detach().numpy()
                )

model_name = 'convlstm'
tensorboard_path = '../tbd'

# def main(model_name, tensorboard_path):
train_loader, valid_loader, test_loader = prepare_data_loaders()
if model_name == 'persistant':
    # score on valid set: 197.64139689523992
    # score on test set: 283.66210850104176
    model = PersistantModel()
elif model_name == 'convlstm':
    model = ConvLSTMModel()
    trainer = L.Trainer(
        logger=L.pytorch.loggers.TensorBoardLogger(save_dir=tensorboard_path),
        max_epochs=1
    )
    trainer.fit(model, train_loader)
else:
    print('Unknown model name')

    # return model


# model = main(model_name, tensorboard_path)

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--model')
#     parser.add_argument('--tensorboard_path', default='../tbd')
#     args = parser.parse_args()
#     main(args.model, args.tensorboard_path)


/content
/content/drive/MyDrive/ColabNotebooks/MLCup2023Weather/sources


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 7 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUD

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
  2%|▏         | 418/16976 [11:54<7:51:55,  1.71s/it]


KeyboardInterrupt: ignored

NameError: ignored

In [ ]:
i = 0
sdfile= 'sdfile.pt'
while True:
    if os.path.exists(sdfile):
        i += 1
        ofc = sdfile.removesuffix('.pt')
        sdfile = f"{ofc}{str(i)}.pt"
    else:
        break

model.state_dict.save('sdfile.pt')

In [ ]:
print(evaluate_on_val(model, valid_loader))
process_test(model, test_loader)
print(evaluate_on_val(model, test_loader))
